# Creating SEC 10-Q Dataset

This notebook takes the Docugami KG-RAG Dataset for SEC 10-Q maintained here: https://github.com/docugami/KG-RAG-datasets/tree/main/sec-10-q and converts it into the correct format for the llama_dataset repo.

Instructions adapted from https://github.com/run-llama/llama_index/blob/main/docs/examples/llama_dataset/ragdataset_submission_template.ipynb

In [1]:
! pip install --quiet --upgrade llama-index pypdf

## Get Latest Dataset artifacts

In [2]:
! rm -rf temp
! git clone https://github.com/docugami/KG-RAG-datasets.git temp

Cloning into 'temp'...
remote: Enumerating objects: 178, done.
remote: Counting objects: 100% (71/71), done.
remote: Compressing objects: 100% (54/54), done.
remote: Total 178 (delta 10), reused 60 (delta 8), pack-reused 107
Receiving objects: 100% (178/178), 56.52 MiB | 47.37 MiB/s, done.
Resolving deltas: 100% (11/11), done.


In [3]:
!rm -rf source_files
!cp -r temp/sec-10-q/data/v1/docs source_files

## Create a LabelledRagDataset

In [4]:
import os
from pathlib import Path

DOCS_DIR = Path(os.getcwd()) / "temp/sec-10-q/data/v1/docs" 
QNA_CSV = Path(os.getcwd()) / "temp/sec-10-q/data/v1/qna_data.csv"

In [5]:
import pandas as pd
from llama_index.llama_dataset import (
    LabelledRagDataExample,
    CreatedBy,
    CreatedByType,
)
from llama_index.llama_dataset import LabelledRagDataset


# Read
source_df = pd.read_csv(QNA_CSV)

examples = []
for ix, row in source_df.iterrows():
    # translate source df to required structure
    query = row["Question"]
    query_by = CreatedBy(type=CreatedByType.HUMAN)
    reference_answer = row["Answer"]
    reference_answer_by = CreatedBy(
        model_name="gpt-4-turbo (with human review)", type=CreatedByType.AI
    )
    reference_contexts = None

    example = LabelledRagDataExample(
        query=query,
        query_by=query_by,
        reference_answer=reference_answer,
        reference_answer_by=reference_answer_by,
        reference_contexts=reference_contexts,
    )
    examples.append(example)

rag_dataset = LabelledRagDataset(examples=examples)

# save this dataset as it is required for the submission
rag_dataset.save_json("rag_dataset.json")

## Generate a Baseline Evaluation Result

In [6]:
from llama_index.readers import SimpleDirectoryReader
from llama_index import VectorStoreIndex
from llama_index.llama_pack import download_llama_pack

documents = SimpleDirectoryReader(input_dir=str(DOCS_DIR)).load_data()

index = VectorStoreIndex.from_documents(documents=documents)
query_engine = index.as_query_engine()

RagEvaluatorPack = download_llama_pack("RagEvaluatorPack", "./pack")
rag_evaluator = RagEvaluatorPack(
    query_engine=query_engine,
    rag_dataset=rag_dataset,
)
benchmark_df = await rag_evaluator.arun()
benchmark_df

Batch processing of predictions: 100%|██████████| 5/5 [00:06<00:00,  1.25s/it]
Batch processing of evaluations: 100%|██████████| 98/98.0 [13:50<00:00,  8.48s/it]
pack/base.py:254: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  mean_scores_df = pd.concat(


rag,base_rag
metrics,
mean_correctness_score,2.666667
mean_relevancy_score,0.825641
mean_faithfulness_score,0.882051
mean_context_similarity_score,NaN


## Prepare card.json and README.md

In [7]:
from llama_index.llama_pack import download_llama_pack

LlamaDatasetMetadataPack = download_llama_pack("LlamaDatasetMetadataPack", "./pack")

metadata_pack = LlamaDatasetMetadataPack()

dataset_description = (
    "A labelled RAG dataset with SEC 10-Q documents for major tech companies including queries "
    "across multiple docs and chunks, with reference answers. "
    "See https://github.com/docugami/KG-RAG-datasets for details."
)

# This creates and saves a card.json and README.md to the same
# directory where you're running this notebook.
metadata_pack.run(
    name="Docugami KG-RAG - SEC 10-Q",
    description=dataset_description,
    rag_dataset=rag_dataset,
    index=index,
    benchmark_df=benchmark_df,
    baseline_name="llamaindex",
)